In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from io import StringIO
df = pd.read_csv('sample.csv')
df.dropna(subset=['Installation Date'], inplace=True)
df.fillna(0, inplace=True)

# --- بخش ۱: مهندسی ویژگی‌های کلیدی برای تحلیل عملکرد ---
# این ویژگی‌ها به ما اجازه می‌دهند تا سلامت محصول را به صورت کمی اندازه‌گیری کنیم.
print("--- ۱. ساخت ویژگی‌های مهندسی برای تحلیل عملکرد محصول ---")
def jalali_to_gregorian_approx(jalali_str):
    try:
        parts = list(map(int, str(jalali_str).split('/')))
        if len(parts) == 3 and 1300 < parts[0] < 1500 and 0 < parts[1] < 13 and 0 < parts[2] < 32:
            return pd.Timestamp(year=parts[0] + 621, month=parts[1], day=parts[2])
    except (ValueError, IndexError):
        pass
    return pd.NaT
df['Installation_Date_G'] = df['Installation Date'].apply(jalali_to_gregorian_approx)
df.dropna(subset=['Installation_Date_G'], inplace=True)
df['Meter_Age_Days'] = (pd.to_datetime('2025-06-11') - df['Installation_Date_G']).dt.days # Using current date for age calculation

total_hours_in_month = 31 * 24
df['Uptime_Percentage'] = np.clip((df['Operating Hours (h)'] / total_hours_in_month) * 100, 0, 100)
df['DQI'] = 0.6 * (df['درصد اطلاعات موجود'] / 100) + 0.4 * (df['Uptime_Percentage'] / 100)
print("ویژگی‌های 'سن کنتور' و 'شاخص کیفیت داده (DQI)' با موفقیت محاسبه شدند.\n")


# --- بخش ۲: تحلیل رابطه عمر محصول با قابلیت اطمینان آن ---
print("--- ۲. تحلیل تأثیر 'سن کنتور' بر عملکرد و قابلیت اطمینان ---")
# حذف کنتورهای تازه نصب شده برای تحلیل بهتر
df_aged = df[df['Meter_Age_Days'] > 30].copy()
correlation = df_aged['Meter_Age_Days'].corr(df_aged['DQI'])
print(f"ضریب همبستگی پیرسون بین سن کنتور و شاخص کیفیت داده (DQI): {correlation:.3f}")

# دسته‌بندی کنتورها بر اساس سن
age_bins = [0, 365, 730, 1095, np.inf]
age_labels = ['< 1 سال', '1-2 سال', '2-3 سال', '> 3 سال']
df_aged['Age_Group'] = pd.cut(df_aged['Meter_Age_Days'], bins=age_bins, labels=age_labels, right=False)
dqi_by_age = df_aged.groupby('Age_Group')['DQI'].mean().reset_index()
print("\nمیانگین شاخص کیفیت داده (DQI) به تفکیک گروه سنی کنتور:")
print(dqi_by_age.round(3))

# مصورسازی با Plotly
fig = px.scatter(df_aged, 
                x='Meter_Age_Days', 
                y='DQI',
                trendline="ols",
                labels={
                    'Meter_Age_Days': 'سن کنتور (روز)',
                    'DQI': 'شاخص کیفیت داده (DQI)'
                },
                title='رابطه بین سن کنتور و شاخص کیفیت داده (DQI)')

fig.update_layout(
    font_family="Tahoma",
    title_font_size=20,
    title_x=0.5,
    plot_bgcolor='white',
    paper_bgcolor='white'
)

fig.update_traces(
    marker=dict(size=8, opacity=0.6),
    selector=dict(mode='markers')
)

fig.show()


# --- بخش ۳: ریشه‌یابی دلایل اصلی افت عملکرد ---
print("\n--- ۳. ریشه‌یابی دلایل اصلی افت عملکرد در کنتورهای ضعیف ---")
df_low_dqi = df[df['DQI'] < 0.5].copy() # تعریف کنتور ضعیف: DQI کمتر از 50%

def get_failure_mode(row):
    # اگر درصد اطلاعات موجود پایین باشد اما آپتایم بالا باشد، مشکل از ارتباطات است
    if row['درصد اطلاعات موجود'] < 50 and row['Uptime_Percentage'] > 50:
        return 'خطای ارتباطی (GPRS/SIM)'
    # اگر آپتایم پایین باشد، مشکل از سخت‌افزار یا باتری است
    if row['Uptime_Percentage'] < 10: # آستانه سخت‌گیرانه برای خرابی قطعی
        return 'خطای سخت‌افزار/باتری'
    return 'ترکیبی / نامشخص'

if not df_low_dqi.empty:
    df_low_dqi['Failure_Mode'] = df_low_dqi.apply(get_failure_mode, axis=1)
    failure_counts = df_low_dqi['Failure_Mode'].value_counts()
    print("تحلیل دلایل خرابی برای کنتورهای با DQI پایین:")
    print(failure_counts)

    # مصورسازی دلایل خرابی با Plotly
    fig = go.Figure(data=[go.Pie(
        labels=failure_counts.index,
        values=failure_counts.values,
        textinfo='label+percent',
        textposition='inside',
        marker=dict(colors=['#ff9999','#66b3ff','#99ff99'])
    )])

    fig.update_layout(
        title='تفکیک دلایل اصلی افت عملکرد در کنتورهای ضعیف',
        font_family="Tahoma",
        title_font_size=20,
        title_x=0.5
    )

    fig.show()
else:
    print("هیچ کنتور با عملکرد ضعیف (DQI < 0.5) برای ریشه‌یابی یافت نشد.")


# --- بخش ۴: تحلیل عملکرد بر اساس بچ تولیدی (با تقریب تاریخ نصب) ---
print("\n--- ۴. تحلیل عملکرد کنتورها بر اساس تاریخ نصب (به عنوان نماینده بچ تولیدی) ---")
df['Installation_Month'] = df['Installation_Date_G'].dt.to_period('M')
dqi_by_batch = df.groupby('Installation_Month')['DQI'].agg(['mean', 'count']).reset_index()
dqi_by_batch.rename(columns={'mean': 'میانگین DQI', 'count': 'تعداد نصب'}, inplace=True)
print("میانگین کیفیت داده به تفکیک ماه نصب:")
print(dqi_by_batch.sort_values('Installation_Month'))

# شناسایی دوره‌های نصب با عملکرد ضعیف
problematic_batches = dqi_by_batch[dqi_by_batch['میانگین DQI'] < 0.6] # آستانه برای عملکرد ضعیف
if not problematic_batches.empty:
    print("\nهشدار: دوره‌های نصب زیر عملکرد ضعیف‌تری نسبت به میانگین داشته‌اند:")
    print(problematic_batches)
else:
    print("\nهیچ دوره نصب با عملکرد به شدت ضعیف شناسایی نشد.")

# --- بخش ۵: خلاصه مدیریتی برای مدیرعامل پویاک ---
print("\n" + "="*60)
print("خلاصه مدیریتی: گزارش عملکرد و قابلیت اطمینان محصول POOYAB-SM4")
print("="*60)
print("این گزارش به منظور ارزیابی سلامت محصولات شرکت در میدان تهیه شده است.\n")

# یافته ۱: عمر محصول
corr_text = f"ضریب همبستگی {correlation:.3f}"
if correlation < -0.5:
    corr_desc = "یک همبستگی منفی قوی را نشان می‌دهد. به این معنی که با افزایش عمر کنتور، عملکرد آن به شکل محسوسی کاهش می‌یابد."
elif correlation < -0.2:
    corr_desc = "یک همبستگی منفی متوسط را نشان می‌دهد. محصولات با گذشت زمان دچار افت عملکرد می‌شوند."
else:
    corr_desc = "همبستگی قوی مشاهده نشد، اما تحلیل گروه‌های سنی همچنان مهم است."
print(f"یافته ۱ (عمر محصول): تحلیل داده‌ها ({corr_text}) {corr_desc}")
print(f"    - پیشنهاد: تدوین سیاست گارانتی و برنامه تعویض پیشگیرانه برای کنتورهای با عمر بیش از {age_labels[-2]}.")

# یافته ۲: دلایل خرابی
if not df_low_dqi.empty:
    dominant_failure = failure_counts.idxmax()
    dominant_percentage = (failure_counts.max() / failure_counts.sum()) * 100
    print(f"\nیافته ۲ (ریشه‌یابی خرابی): شایع‌ترین حالت خرابی در کنتورهای ضعیف، '{dominant_failure}' با سهم {dominant_percentage:.1f}% است.")
    print("    - پیشنهاد: تمرکز تیم تحقیق و توسعه (R&D) بر روی بهبود ماژول‌های ارتباطی و آنتن‌دهی در نسل بعدی محصول.")
else:
    print("\nیافته ۲ (ریشه‌یابی خرابی): در این مجموعه داده، تعداد کنتورهای با عملکرد بسیار ضعیف برای نتیجه‌گیری قطعی کافی نبود.")

# یافته ۳: کنترل کیفیت
if not problematic_batches.empty:
    batch_dates = ', '.join(problematic_batches['Installation_Month'].astype(str).tolist())
    print(f"\nیافته ۳ (کنترل کیفیت): کنتورهای نصب‌شده در ماه‌های {batch_dates} عملکرد پایین‌تری داشته‌اند.")
    print("    - پیشنهاد: بررسی سوابق تولید و تیم‌های نصب مربوط به این دوره‌ها برای شناسایی مشکلات احتمالی در کنترل کیفیت.")
else:
    print("\nیافته ۳ (کنترل کیفیت): تفاوت چشمگیری در عملکرد بچ‌های مختلف نصب مشاهده نشد.")
print("\n" + "="*60 + "\n")

--- ۱. ساخت ویژگی‌های مهندسی برای تحلیل عملکرد محصول ---
ویژگی‌های 'سن کنتور' و 'شاخص کیفیت داده (DQI)' با موفقیت محاسبه شدند.

--- ۲. تحلیل تأثیر 'سن کنتور' بر عملکرد و قابلیت اطمینان ---
ضریب همبستگی پیرسون بین سن کنتور و شاخص کیفیت داده (DQI): 0.631

میانگین شاخص کیفیت داده (DQI) به تفکیک گروه سنی کنتور:
  Age_Group    DQI
0   < 1 سال  0.000
1   1-2 سال  0.071
2   2-3 سال  0.021
3   > 3 سال  0.734


/tmp/ipykernel_80000/3109964279.py:46: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  dqi_by_age = df_aged.groupby('Age_Group')['DQI'].mean().reset_index()



--- ۳. ریشه‌یابی دلایل اصلی افت عملکرد در کنتورهای ضعیف ---
تحلیل دلایل خرابی برای کنتورهای با DQI پایین:
Failure_Mode
خطای سخت‌افزار/باتری    126
Name: count, dtype: int64



--- ۴. تحلیل عملکرد کنتورها بر اساس تاریخ نصب (به عنوان نماینده بچ تولیدی) ---
میانگین کیفیت داده به تفکیک ماه نصب:
   Installation_Month  میانگین DQI  تعداد نصب
0             2019-08     0.000000          5
1             2020-03     0.000000          1
2             2020-05     1.000000          3
3             2020-06     0.800000          2
4             2020-07     0.666667          9
5             2020-08     0.200000          3
6             2020-09     0.174180          1
7             2021-02     0.860929          9
8             2021-03     0.872043          6
9             2021-04     0.890909         11
10            2021-05     1.000000          1
11            2021-06     0.999758          6
12            2021-07     0.851619          9
13            2021-08     0.941799         11
14            2021-09     0.771505          5
15            2021-11     1.000000          1
16            2022-04     0.000000          1
17            2022-05     0.600000          1
18       

In [5]:
# --- بخش ۱: ساخت مطالعه موردی (Case Study) برای کاهش آب بدون درآمد (NRW) ---
print("--- ۱. شناسایی نقاط مشکوک به هدررفت آب جهت کاهش آب بدون درآمد (NRW) ---")
print("هدف: شناسایی مشترکینی که مصرفشان به طور غیرعادی بالاتر از همتایانشان است.")

# برای تحلیل دقیق‌تر، فقط کنتورهای با داده سالم را در نظر می‌گیریم
df_healthy = df[df['درصد اطلاعات موجود'] > 90].copy()

# تعریف گروه‌های همتا بر اساس نوع پروانه و سایز کنتور
df_healthy['Peer_Group'] = df_healthy['License Type'] + ' - ' + df_healthy['Meter Size'].astype(int).astype(str) + 'mm'

anomalies_list = []
# استفاده از روش IQR برای شناسایی داده‌های پرت در هر گروه همتا
for group_name, group_data in df_healthy.groupby('Peer_Group'):
    if len(group_data) < 5: continue # از گروه‌های کوچک صرف‌نظر می‌کنیم

    q1 = group_data['Consumption (m³)'].quantile(0.25)
    q3 = group_data['Consumption (m³)'].quantile(0.75)
    iqr = q3 - q1
    upper_bound = q3 + 1.5 * iqr
    
    # شناسایی ناهنجاری (مصرف بالاتر از حد انتظار)
    anomalies = group_data[group_data['Consumption (m³)'] > upper_bound]
    if not anomalies.empty:
        for index, row in anomalies.iterrows():
            anomalies_list.append({
                'Subscription Code': row['Subscription Code'],
                'Peer_Group': row['Peer_Group'],
                'Actual_Consumption': row['Consumption (m³)'],
                'Peer_Group_Median_Consumption': group_data['Consumption (m³)'].median(),
                'Excess_Consumption': row['Consumption (m³)'] - group_data['Consumption (m³)'].median()
            })

if anomalies_list:
    anomaly_df = pd.DataFrame(anomalies_list)
    TARIFF_PER_M3_ANNUAL = 5000 * 12 # تعرفه فرضی سالانه
    anomaly_df['Potential_Annual_Loss_Rials'] = anomaly_df['Excess_Consumption'] * TARIFF_PER_M3_ANNUAL
    
    print("\nگزارش مطالعه موردی: شناسایی پتانسیل‌های کاهش NRW")
    print("کنتورهای زیر مصرف بسیار بالاتری نسبت به گروه همتای خود دارند و کاندیدای بررسی نشت یا مصرف غیرمجاز هستند:")
    print(anomaly_df.sort_values('Potential_Annual_Loss_Rials', ascending=False).round(2))
else:
    print("\nدر این مجموعه داده، ناهنجاری مصرف قابل توجهی شناسایی نشد.")

# مصورسازی: نمایش توزیع مصرف در چند گروه همتای کلیدی برای نشان دادن نقاط پرت با Plotly
major_groups = df_healthy['Peer_Group'].value_counts().nlargest(3).index
df_filtered = df_healthy[df_healthy['Peer_Group'].isin(major_groups)]

fig = px.box(df_filtered, 
             x='Consumption (m³)', 
             y='Peer_Group',
             title='توزیع مصرف در گروه‌های همتا و شناسایی نقاط پرت (پتانسیل NRW)')

fig.update_layout(
    font_family="Tahoma",
    title_font_size=20,
    title_x=0.5,
    plot_bgcolor='white',
    paper_bgcolor='white',
    yaxis_title='گروه همتا (نوع پروانه و سایز کنتور)',
    xaxis_title='مصرف ماهانه (m³)'
)

fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='lightgray')
fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='lightgray')

fig.show()


# --- بخش ۲: نمایش ارزش ویژگی "شناسایی جریان منفی" ---
print("\n\n--- ۲. نمایش ارزش کنتور پویاک به عنوان ابزار تشخیص سلامت شبکه ---")
print("هدف: تبدیل ویژگی فنی 'جریان منفی' به یک هشدار عملیاتی ارزشمند برای مشتری.")

df['Negative_Flow_Flag'] = (df['Number of Negative Flows'] > 0).astype(int)
negative_flow_meters = df[df['Negative_Flow_Flag'] == 1]

if not negative_flow_meters.empty:
    print("\nهشدار تشخیصی سلامت شبکه برای تیم بهره‌برداری مشتری:")
    for index, row in negative_flow_meters.iterrows():
        print("-" * 50)
        print(f"کنتور با سریال: {row['Meter Serial']:.0f} (اشتراک: {row['Subscription Code']})")
        print(f"تعداد رویداد جریان معکوس ثبت شده: {row['Number of Negative Flows']:.0f} بار")
        print("\nتوصیه عملیاتی:")
        print("  این هشدار لزوماً خطای کنتور نیست، بلکه یک سیگنال تشخیصی ارزشمند است.")
        print("  بر اساس اسناد فنی، این پدیده می‌تواند ناشی از مشکلات هیدرولیکی در شبکه (مانند نوسان فشار یا خرابی شیر یکطرفه) باشد.")
        print("  **پیشنهاد می‌شود یک بازرسی میدانی برای ریشه‌یابی علت این پدیده انجام شود.**")
        print("  این قابلیت تشخیصی، یکی از مزایای رقابتی کنتور POOYAB-SM4 است.")
        print("-" * 50)
else:
    print("\nدر این مجموعه داده، هیچ کنتوری با سابقه جریان منفی یافت نشد.")
    print("سیستم تحلیلی پویاک آماده است تا به محض ثبت چنین رویدادی، هشدارهای تشخیصی لازم را صادر کند.")


# --- بخش ۳: خلاصه مدیریتی برای تیم فروش و بازاریابی پویاک ---
print("\n" + "="*70)
print("خلاصه اجرایی برای تیم فروش: چگونه ارزش محصول پویاک را به مشتری بفروشیم؟")
print("="*70)
print("این تحلیل، داده‌ها را به دو ابزار فروش قدرتمند تبدیل می‌کند:\n")

print("ابزار فروش ۱: ارائه گزارش کاهش آب بدون درآمد (NRW)")
print("-" * 55)
if anomalies_list:
    top_anomaly = anomaly_df.sort_values('Potential_Annual_Loss_Rials', ascending=False).iloc[0]
    print("گفتگوی نمونه با مشتری:")
    print(f"  'بر اساس تحلیل ما از داده‌هایتان، کنتور اشتراک {top_anomaly['Subscription Code']} به تنهایی در ماه گذشته {top_anomaly['Excess_Consumption']:.0f} متر مکعب بیشتر از همتایان خود مصرف کرده است.")
    print(f"   این مصرف مازاد می‌تواند سالانه هزینه‌ای بالغ بر {top_anomaly['Potential_Annual_Loss_Rials']:,.0f} ریال به دلیل نشت یا مصرف غیرمجاز به شما تحمیل کند.")
    print("   ما به شما کمک می‌کنیم تا این نقاط را به سرعت شناسایی و منابع خود را بهینه مدیریت کنید.'\n")
else:
    print("  (در این دیتاست ناهنجاری مشخصی برای ساخت نمونه یافت نشد، اما متدولوژی آماده است.)\n")

print("ابزار فروش ۲: نمایش کنتور به عنوان سنسور سلامت شبکه")
print("-" * 55)
print("گفتگوی نمونه با مشتری:")
if not negative_flow_meters.empty:
    top_neg_flow = negative_flow_meters.iloc[0]
    print(f"  'کنتور ما برای اشتراک {top_neg_flow['Subscription Code']}، چندین مورد جریان معکوس را ثبت کرده است. این یک قابلیت تشخیصی منحصر به فرد است.")
    print("   این یعنی محصول ما فراتر از یک ابزار صورتحساب، یک سنسور هوشمند برای پایش سلامت شبکه شماست و به شما در شناسایی مشکلات پنهان کمک می‌کند.'\n")
else:
    print("  'محصول ما قابلیت شناسایی جریان معکوس را دارد. این ویژگی به شما کمک می‌کند تا مشکلات هیدرولیکی شبکه را قبل از اینکه جدی شوند، تشخیص دهید.")
    print("   این یعنی کنتور ما یک سرمایه‌گذاری در جهت پایداری و سلامت کل شبکه شماست.'\n")

print("پیام نهایی: ما سخت‌افزار نمی‌فروشیم، ما **هوش عملیاتی (Actionable Intelligence)** می‌فروشیم.")
print("="*70)

--- ۱. شناسایی نقاط مشکوک به هدررفت آب جهت کاهش آب بدون درآمد (NRW) ---
هدف: شناسایی مشترکینی که مصرفشان به طور غیرعادی بالاتر از همتایانشان است.

گزارش مطالعه موردی: شناسایی پتانسیل‌های کاهش NRW
کنتورهای زیر مصرف بسیار بالاتری نسبت به گروه همتای خود دارند و کاندیدای بررسی نشت یا مصرف غیرمجاز هستند:
  Subscription Code               Peer_Group  Actual_Consumption  \
4            515530          کشاورزی - 150mm           248740.01   
2            514579          کشاورزی - 100mm           193203.30   
1            517638          کشاورزی - 100mm           113348.60   
5            603020          کشاورزی - 150mm           129221.48   
3            514505          کشاورزی - 125mm            54595.60   
0            911593  شرب و بهداشت دام - 50mm            11507.03   

   Peer_Group_Median_Consumption  Excess_Consumption  \
4                       39231.70           209508.31   
2                       17073.53           176129.77   
1                       17073.53            96275.07  



--- ۲. نمایش ارزش کنتور پویاک به عنوان ابزار تشخیص سلامت شبکه ---
هدف: تبدیل ویژگی فنی 'جریان منفی' به یک هشدار عملیاتی ارزشمند برای مشتری.

هشدار تشخیصی سلامت شبکه برای تیم بهره‌برداری مشتری:
--------------------------------------------------
کنتور با سریال: 909801034997 (اشتراک: 514059)
تعداد رویداد جریان معکوس ثبت شده: 21 بار

توصیه عملیاتی:
  این هشدار لزوماً خطای کنتور نیست، بلکه یک سیگنال تشخیصی ارزشمند است.
  بر اساس اسناد فنی، این پدیده می‌تواند ناشی از مشکلات هیدرولیکی در شبکه (مانند نوسان فشار یا خرابی شیر یکطرفه) باشد.
  **پیشنهاد می‌شود یک بازرسی میدانی برای ریشه‌یابی علت این پدیده انجام شود.**
  این قابلیت تشخیصی، یکی از مزایای رقابتی کنتور POOYAB-SM4 است.
--------------------------------------------------
--------------------------------------------------
کنتور با سریال: 909801036264 (اشتراک: 515122)
تعداد رویداد جریان معکوس ثبت شده: 1 بار

توصیه عملیاتی:
  این هشدار لزوماً خطای کنتور نیست، بلکه یک سیگنال تشخیصی ارزشمند است.
  بر اساس اسناد فنی، این پدیده می‌تواند ناشی 

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
# اجرای مهندسی ویژگی
def jalali_to_gregorian_approx(jalali_str):
    try:
        parts = list(map(int, str(jalali_str).split('/')))
        if len(parts) == 3 and 1300 < parts[0] < 1500 and 0 < parts[1] < 13 and 0 < parts[2] < 32:
            return pd.Timestamp(year=parts[0] + 621, month=parts[1], day=parts[2])
    except: return pd.NaT
df['Installation_Date_G'] = df['Installation Date'].apply(jalali_to_gregorian_approx)
df.dropna(subset=['Installation_Date_G'], inplace=True)
df['Meter_Age_Days'] = (pd.to_datetime('2025-06-11') - df['Installation_Date_G']).dt.days
df['DQI'] = 0.6 * (df['درصد اطلاعات موجود'] / 100) + 0.4 * (np.clip((df['Operating Hours (h)'] / (31*24)) * 100, 0, 100) / 100)

# اجرای خوشه‌بندی
df_healthy = df[df['DQI'] > 0.5].copy()
if len(df_healthy) > 3: # K-Means needs at least n_clusters samples
    features_for_clustering = df_healthy[['Consumption (m³)', 'Meter Size', 'DQI', 'Meter_Age_Days']]
    scaler = StandardScaler()
    features_scaled = scaler.fit_transform(features_for_clustering)
    kmeans = KMeans(n_clusters=3, random_state=42, n_init=10)
    df_healthy['Cluster'] = kmeans.fit_predict(features_scaled)
    # Drop any existing Cluster column in df before merge to avoid duplicates
    if 'Cluster' in df.columns:
        df = df.drop('Cluster', axis=1)
    df = df.merge(df_healthy[['Subscription Code', 'Cluster']], on='Subscription Code', how='left')


# --- بخش ۱: شناسایی و پروفایل‌سازی سگمنت‌های استراتژیک بازار ---
print("--- ۱. هوش تجاری: پروفایل‌سازی بخش‌های مختلف بازار ---")
print("هدف: شناسایی ارزشمندترین گروه‌های مشتری برای تمرکز استراتژیک تیم فروش.")

if 'Cluster' in df.columns:
    # تعریف پرسونای هر خوشه
    cluster_profiles = {}
    for i in range(3): # Assuming 3 clusters
        cluster_data = df_healthy[df_healthy['Cluster'] == i]
        profile = {
            'avg_consumption': cluster_data['Consumption (m³)'].mean(),
            'avg_dqi': cluster_data['DQI'].mean(),
            'avg_meter_size': cluster_data['Meter Size'].mean(),
            'dominant_license': cluster_data['License Type'].mode()[0] if not cluster_data.empty else 'N/A',
            'count': len(cluster_data)
        }
        # نام‌گذاری هوشمند بر اساس ویژگی‌ها
        if profile['avg_consumption'] > df_healthy['Consumption (m³)'].mean() and profile['avg_dqi'] > 0.9:
            persona = "ستاره طلایی (مشتریان کلیدی و وفادار)"
        elif profile['avg_dqi'] < 0.7:
            persona = "نگهداری پرهزینه (نیازمند توجه فنی)"
        else:
            persona = "پتانسیل رشد (فرصت‌های Upsell)"
        cluster_profiles[i] = {'persona': persona, 'profile': profile}

    print("\nپروفایل سگمنت‌های مشتریان کشف شده توسط هوش مصنوعی:")
    for cid, data in cluster_profiles.items():
        print("-" * 40)
        print(f"خوشه {cid}: {data['persona']}")
        print(f"  - تعداد اعضا: {data['profile']['count']}")
        print(f"  - میانگین مصرف: {data['profile']['avg_consumption']:.0f} m³")
        print(f"  - میانگین کیفیت داده: {data['profile']['avg_dqi']:.2f}")
        print(f"  - نوع پروانه غالب: {data['profile']['dominant_license']}")
        print(f"  - توصیه استراتژیک:")
        if "ستاره طلایی" in data['persona']:
            print("      حفظ این مشتریان و ارائه خدمات ویژه به آنها در اولویت است.")
        elif "نگهداری پرهزینه" in data['persona']:
            print("      تیم فنی باید برای بهبود DQI این گروه اقدام کند تا از زیان جلوگیری شود.")
        else:
            print("      تیم فروش می‌تواند با ارائه خدمات جانبی، درآمد از این بخش را افزایش دهد.")
else:
    print("\nداده کافی برای پروفایل‌سازی سگمنت‌ها وجود نداشت.")

# --- بخش ۲: طراحی نمونه اولیه سرویس "تحلیل داده به عنوان خدمت" (AaaS) ---
print("\n\n--- ۲. فرصت درآمدی جدید: نمونه اولیه گزارش 'Analytics as a Service' ---")
print("هدف: طراحی یک محصول داده‌محور برای فروش به مشتریان به عنوان یک سرویس اشتراکی.")

def generate_customer_report(customer_code):
    customer_data = df[df['Subscription Code'] == customer_code].iloc[0]
    
    print("\n" + "="*70)
    print("      گزارش ماهیانه سلامت و بهره‌وری کنتور - ویژه مشترک")
    print(f"                   اشتراک شماره: {customer_code}")
    print("                  تهیه شده توسط پلتفرم هوشمند پویاک")
    print("="*70)

    # بخش A: عملکرد و قابلیت اطمینان کنتور
    print("\nبخش الف: کارت امتیازی عملکرد کنتور شما")
    print(f"  - شاخص کیفیت داده (DQI): {customer_data['DQI']:.2%} (نشان‌دهنده سلامت کلی ارتباط و سخت‌افزار)")
    if customer_data['DQI'] > 0.9:
        print("    وضعیت: عالی. کنتور شما با بالاترین قابلیت اطمینان در حال کار است.")
    elif customer_data['DQI'] > 0.7:
        print("    وضعیت: خوب. عملکرد کنتور شما پایدار است.")
    else:
        print("    وضعیت: نیازمند توجه. ممکن است در ارسال داده اختلالاتی وجود داشته باشد.")

    # بخش B: هشدار پتانسیل هدررفت آب
    print("\nبخش ب: تحلیل مقایسه‌ای مصرف (پتانسیل نشت یا هدررفت)")
    # این بخش می‌تواند با تحلیل ناهنجاری از محور ۲ تکمیل شود
    print("  (در این نمونه اولیه، این بخش برای نمایش مفهوم طراحی شده است.)")
    print("  - تحلیل‌های پیشرفته پویاک می‌تواند مصرف شما را با همتایانتان مقایسه کرده و هشدارهای مربوط به مصرف غیرعادی را صادر کند.")
    
    # بخش C: هشدار سلامت شبکه
    print("\nبخش پ: گزارش رویدادهای خاص شبکه")
    if customer_data['Number of Negative Flows'] > 0:
        print(f"  - هشدار: {int(customer_data['Number of Negative Flows'])} مورد جریان معکوس در کنتور شما ثبت شده است.")
        print("    این یک قابلیت تشخیصی است که می‌تواند نشانگر مشکلات هیدرولیکی در شبکه داخلی شما باشد.")
    else:
        print("  - در این ماه هیچ رویداد جریان معکوس ثبت نشده است.")
        
    print("\n" + "-"*70)
    print("این گزارش نمونه‌ای از 'سرویس تحلیلی پرمیوم پویاک' است.")
    print("برای دریافت گزارش‌های ماهانه و دسترسی به داشبورد تحلیلی با ما تماس بگیرید.")
    print("="*70)

# نمایش گزارش برای یک مشتری نمونه (مثلاً از خوشه ستاره طلایی)
if 'Cluster' in df.columns:
    golden_star_cluster_id = [cid for cid, data in cluster_profiles.items() if 'ستاره طلایی' in data['persona']]
    if golden_star_cluster_id:
        sample_customer = df[df['Cluster'] == golden_star_cluster_id[0]].iloc[0]['Subscription Code']
        generate_customer_report(sample_customer)

# --- بخش ۳: خلاصه مدیریتی برای مدیرعامل پویاک ---
print("\n" + "="*70)
print("خلاصه اجرایی برای مدیرعامل: چگونه از داده برای رشد شرکت استفاده کنیم؟")
print("="*70)

print("فرصت ۱: تمرکز هوشمند در بازار")
print("-" * 40)
print("  - تحلیل ما به طور خودکار پرسودترین و وفادارترین بخش‌های بازار را شناسایی می‌کند.")
if 'Cluster' in df.columns:
    print(f"  - پیشنهاد مشخص: تمرکز تیم فروش بر روی مشتریان با پروفایل مشابه خوشه '{cluster_profiles[golden_star_cluster_id[0]]['persona']}'، که مشخصاً '{cluster_profiles[golden_star_cluster_id[0]]['profile']['dominant_license']}' هستند، می‌تواند نرخ موفقیت را به شدت افزایش دهد.")
else:
     print("  - با داده‌های کامل‌تر، می‌توانیم پروفایل مشتری ایده‌آل (ICP) را استخراج کنیم.")


print("\nفرصت ۲: خلق جریان درآمدی جدید و پایدار")
print("-" * 40)
print("  - ما صرفاً یک شرکت سخت‌افزاری نیستیم. ما یک شرکت داده‌محور هستیم.")
print("  - نمونه اولیه گزارش 'Analytics as a Service' نشان می‌دهد که ما محصولی ارزشمند برای فروش داریم که فراتر از کنتور است.")
print("  - پیشنهاد مشخص: اجرای یک برنامه پایلوت با ۳ الی ۵ مشتری کلیدی برای ارائه این سرویس تحلیلی به صورت اشتراکی.")
print("    این اقدام، مدل کسب‌وکار ما را به سمت فروش خدمات با حاشیه سود بالا متحول می‌کند.")

print("\nاستراتژی نهایی: تبدیل داده به سود خالص از طریق هوش تجاری.")
print("="*70)

--- ۱. هوش تجاری: پروفایل‌سازی بخش‌های مختلف بازار ---
هدف: شناسایی ارزشمندترین گروه‌های مشتری برای تمرکز استراتژیک تیم فروش.

پروفایل سگمنت‌های مشتریان کشف شده توسط هوش مصنوعی:
----------------------------------------
خوشه 0: پتانسیل رشد (فرصت‌های Upsell)
  - تعداد اعضا: 31
  - میانگین مصرف: 7786 m³
  - میانگین کیفیت داده: 0.98
  - نوع پروانه غالب: کشاورزی
  - توصیه استراتژیک:
      تیم فروش می‌تواند با ارائه خدمات جانبی، درآمد از این بخش را افزایش دهد.
----------------------------------------
خوشه 1: نگهداری پرهزینه (نیازمند توجه فنی)
  - تعداد اعضا: 12
  - میانگین مصرف: 7 m³
  - میانگین کیفیت داده: 0.61
  - نوع پروانه غالب: شرب و بهداشت دام
  - توصیه استراتژیک:
      تیم فنی باید برای بهبود DQI این گروه اقدام کند تا از زیان جلوگیری شود.
----------------------------------------
خوشه 2: ستاره طلایی (مشتریان کلیدی و وفادار)
  - تعداد اعضا: 29
  - میانگین مصرف: 63584 m³
  - میانگین کیفیت داده: 1.00
  - نوع پروانه غالب: کشاورزی
  - توصیه استراتژیک:
      حفظ این مشتریان و ارائه خدمات ویژ

In [10]:
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import pandas as pd

# Calculate total hours in month more accurately
days_in_month = pd.to_datetime('2024-01-31') - pd.to_datetime('2024-01-01')
total_hours = days_in_month.days * 24

df['Uptime_Percentage'] = np.clip((df['Operating Hours (h)'] / total_hours) * 100, 0, 100)
df['DQI'] = 0.6 * (df['درصد اطلاعات موجود'] / 100) + 0.4 * (df['Uptime_Percentage'] / 100)

# --- بخش ۱: استخراج اولویت‌های تحقیق و توسعه (R&D) بر اساس داده‌های میدانی ---
print("--- ۱. نقشه راه محصول (POOYAB-SM5) بر اساس داده‌های عملکردی ---")
print("هدف: تبدیل نقاط ضعف محصول فعلی به نقاط قوت محصول آینده.")

# ۱.۱. بازبینی دلایل اصلی خرابی از محور ۱
df_low_dqi = df[df['DQI'] < 0.5].copy()
def get_failure_mode(row):
    if row['درصد اطلاعات موجود'] < 50 and row['Uptime_Percentage'] > 50: return 'خطای ارتباطی (GPRS/SIM)'
    if row['Uptime_Percentage'] < 10: return 'خطای سخت‌افزار/باتری'
    return 'ترکیبی / نامشخص'

r_and_d_priorities = {}
if not df_low_dqi.empty:
    df_low_dqi['Failure_Mode'] = df_low_dqi.apply(get_failure_mode, axis=1)
    failure_counts = df_low_dqi['Failure_Mode'].value_counts()
    dominant_failure = failure_counts.idxmax()
    r_and_d_priorities[1] = f"بهبود ماژول‌های ارتباطی و آنتن‌دهی (زیرا '{dominant_failure}' شایع‌ترین علت خرابی است)."

# ۱.۲. تحلیل نیاز به آلارم‌های هوشمندتر
# اگر کنتوری مصرف صفر دارد ولی آپتایم و درصد اطلاعات آن 100 است، می‌تواند نشانه دستکاری یا گرفتگی باشد.
suspicious_zero_consumption = df[(df['Consumption (m³)'] == 0) & (df['DQI'] > 0.95)]
if not suspicious_zero_consumption.empty:
    r_and_d_priorities[2] = "توسعه آلارم‌های هوشمند جدید (مانند آلارم 'مصرف صفر با وجود اتصال کامل') برای تشخیص دستکاری یا گرفتگی."

# ۱.۳. تحلیل نیاز به انعطاف‌پذیری نرم‌افزار
unknown_license_count = len(df[df['License Type'] == 'Unknown'])
if unknown_license_count > 0:
    r_and_d_priorities[3] = "افزایش انعطاف‌پذیری نرم‌افزار و پلتفرم برای مدیریت و تعریف 'انواع پروانه' جدید و ناشناخته."

print("\nاولویت‌های تحقیق و توسعه (R&D) پیشنهادی برای کنتور نسل آینده (POOYAB-SM5):")
if r_and_d_priorities:
    for priority, reason in r_and_d_priorities.items():
        print(f"  اولویت {priority}: {reason}")
df_healthy = df[df['DQI'] > 0.5].copy()
cluster_baselines = {}
if len(df_healthy) > 3:
    features_for_clustering = df_healthy[['Consumption (m³)', 'Meter Size', 'DQI', 'Meter_Age_Days']]
    # Handle any NaN values before clustering
    features_for_clustering = features_for_clustering.fillna(features_for_clustering.mean())
# --- بخش ۲: شبیه‌سازی موتور تحلیل پیش‌بینی (Predictive Analytics) - سنگ بنای AMI 2.0 ---
print("\n\n--- ۲. نمونه اولیه موتور پیش‌بینی: گامی به سوی مدیریت هوشمند آب (AMI 2.0) ---")
print("هدف: نمایش قابلیت پلتفرم پویاک در حرکت از گزارش‌دهی گذشته‌نگر به نظارت هوشمند و آینده‌نگر.")

# ۲.۱. ساخت پروفایل مصرف برای هر سگمنت مشتری (با استفاده از خوشه‌بندی)
df_healthy = df[df['DQI'] > 0.5].copy()
cluster_baselines = {}
if len(df_healthy) > 3:
    features_for_clustering = df_healthy[['Consumption (m³)', 'Meter Size', 'DQI']]
    scaler = StandardScaler()
    features_scaled = scaler.fit_transform(features_for_clustering)
    kmeans = KMeans(n_clusters=3, random_state=42, n_init=10)
    df_healthy['Cluster'] = kmeans.fit_predict(features_scaled)
    df = df.merge(df_healthy[['Subscription Code', 'Cluster']], on='Subscription Code', how='left')

    for i in range(3):
        cluster_data = df_healthy[df_healthy['Cluster'] == i]
        cluster_baselines[i] = {
            'mean': cluster_data['Consumption (m³)'].mean(),
            'std': cluster_data['Consumption (m³)'].std()
        }
    print("\nپروفایل‌های مصرف برای هر سگمنت مشتری با موفقیت ساخته شد.")

# ۲.۲. تعریف تابع نظارت پیش‌بینی
def predictive_monitoring_alert(subscription_code, new_consumption):
    meter_info = df[df['Subscription Code'] == subscription_code]
    if meter_info.empty or pd.isna(meter_info.iloc[0]['Cluster']):
        return "هشدار: اطلاعات کافی برای تحلیل پیش‌بینی این مشترک وجود ندارد."
    
    cluster_id = meter_info.iloc[0]['Cluster']
    baseline = cluster_baselines[cluster_id]
    
    if baseline['std'] == 0: return "هشدار: انحراف معیار پروفایل صفر است، تحلیل ممکن نیست."
    
    z_score = (new_consumption - baseline['mean']) / baseline['std']
    
    alert = f"تحلیل پیش‌بینی برای اشتراک {subscription_code} (مصرف جدید: {new_consumption} m³):\n"
    if z_score > 2.0:
        alert += f"  >> هشدار سطح بالا: مصرف این دوره {abs(z_score):.1f} انحراف معیار بالاتر از میانگین پروفایل این مشترک است. احتمال بالای نشت عمده یا تغییر ناگهانی در مصرف."
    elif z_score < -1.5:
        alert += f"  >> هشدار سطح متوسط: مصرف این دوره {abs(z_score):.1f} انحراف معیار پایین‌تر از میانگین پروفایل است. احتمال توقف فعالیت یا دستکاری کنتور."
    else:
        alert += "  >> وضعیت نرمال: مصرف در محدوده قابل انتظار برای این پروفایل مشتری قرار دارد."
    return alert

# ۲.۳. اجرای شبیه‌سازی
if cluster_baselines:
    print("\nاجرای شبیه‌سازی نظارت پیش‌بینی بر روی چند نمونه:")
    sample_customer_1 = df_healthy.iloc[0]['Subscription Code']
    print(predictive_monitoring_alert(sample_customer_1, 3000)) # مصرف نرمال
    print(predictive_monitoring_alert(sample_customer_1, 25000)) # مصرف بسیار بالا
    sample_customer_2 = df_healthy.iloc[1]['Subscription Code']
    print(predictive_monitoring_alert(sample_customer_2, 10)) # مصرف بسیار پایین


# --- بخش ۳: خلاصه مدیریتی برای مدیرعامل پویاک ---
print("\n" + "="*70)
print("خلاصه اجرایی برای مدیرعامل: چشم‌انداز و نقشه راه نوآوری پویاک")
print("="*70)

print("چشم‌انداز محصول (POOYAB-SM5):")
print("-" * 40)
print("  داده‌های میدانی به ما می‌گویند که نسل بعدی کنتور ما باید در سه حوزه کلیدی برتری داشته باشد:")
if r_and_d_priorities:
    for p, r in r_and_d_priorities.items():
        print(f"    {p}. {r}")
else:
    print("    (نیاز به داده‌های بیشتر برای تعیین قطعی اولویت‌ها)")
print("  هدف: ساخت 'قابل اعتمادترین' و 'هوشمندترین' کنتور بازار از طریق تمرکز بر قابلیت اتصال و آلارم‌های تشخیصی پیشرفته.")


print("\nچشم‌انداز پلتفرم (گذار به AMI 2.0):")
print("-" * 40)
print("  ما با موفقیت سنگ بنای یک موتور تحلیل پیش‌بینی را شبیه‌سازی کردیم.")
print("  این قابلیت، مدل کسب‌وکار ما را از 'گزارش‌دهی گذشته' به 'ارائه هوشمندی آینده‌نگر' تغییر می‌دهد.")
print("  پیشنهاد استراتژیک: توسعه این موتور پیش‌بینی به عنوان یک ماژول کلیدی در سرویس AaaS (محور ۳) و بازاریابی پویاک به عنوان یک شرکت 'هوش آبی به عنوان خدمت' (Water Intelligence as a Service - WIaaS).")

print("\nاقدامات پیشنهادی فوری:")
print("  ۱. تخصیص منابع R&D به اولویت‌های استخراج شده از داده‌های میدانی برای طراحی POOYAB-SM5.")
print("  ۲. توسعه ماژول 'نظارت پیش‌بینی' و ارائه آن به صورت آزمایشی به مشتریان کلیدی سرویس AaaS.")
print("  ۳. آغاز تدوین استراتژی بازاریابی جدید با تمرکز بر پویاک به عنوان یک رهبر در تحول دیجیتال صنعت آب.")
print("="*70)

--- ۱. نقشه راه محصول (POOYAB-SM5) بر اساس داده‌های عملکردی ---
هدف: تبدیل نقاط ضعف محصول فعلی به نقاط قوت محصول آینده.

اولویت‌های تحقیق و توسعه (R&D) پیشنهادی برای کنتور نسل آینده (POOYAB-SM5):
  اولویت 1: بهبود ماژول‌های ارتباطی و آنتن‌دهی (زیرا 'خطای سخت‌افزار/باتری' شایع‌ترین علت خرابی است).
  اولویت 3: افزایش انعطاف‌پذیری نرم‌افزار و پلتفرم برای مدیریت و تعریف 'انواع پروانه' جدید و ناشناخته.


--- ۲. نمونه اولیه موتور پیش‌بینی: گامی به سوی مدیریت هوشمند آب (AMI 2.0) ---
هدف: نمایش قابلیت پلتفرم پویاک در حرکت از گزارش‌دهی گذشته‌نگر به نظارت هوشمند و آینده‌نگر.


MergeError: Passing 'suffixes' which cause duplicate columns {'Cluster_x'} is not allowed.